Here are all the data cleaning steps.

1. Get urls from several directories. 
    - The directories were - jasmine, w3newspapers, usnpl
2. Collate all urls together.
3. Extract base urls of all these urls. This creates the "raw dataset".
4. Enrich data of a base url if it is in multiple directories (i.e) if one directory has state info and another has topic info, then make sure the variables is populated in all rows against that url.
5. Dedupe these base urls across directories. This creates the "urls list".
    - To keep track if certain urls exist across states, a separate "collated dataset" is created which has states data and n repeats for n states.
6. Check how many requests of `https://[base url]/ads.txt` are successful. The successful urls form the "valid urls" list
7. Out of the "valid urls" account for redirects.
    - If the base url has a valid redirect to an actual ads.txt file, then that base url enters the "valid redirects" list .
    - If it doesn't lead to an ads.txt directly, but instead to a new base url, then check if that new base url leads to a valid ads.txt. If it does, then create a mapping file from old to new base urls and this new base url should enter the "valid redirects list"

In [1]:
import pandas as pd
import requests as req
from urllib.parse import urlparse
import os
from openpyxl import load_workbook

## 1. Get urls from several directories

Step 1 was done separately. Some of it was manual work. We start from step 2 below.

## 2. Collate all urls together

### 2.1 Tidying up Jasmine directory dataset for collation

In [2]:
#This code is from here: https://stackoverflow.com/a/64759478/10098211
def get_sheetnames_xlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

filepath = "./data/Jasmine directory.xlsx"
sheet_names = get_sheetnames_xlsx(filepath)
dfs_list = []
for sheet in sheet_names:
    df = pd.read_excel(filepath, sheet, header=None, names=['URL'])
    print(sheet, ": ", len(df))
    df['Topic'] = sheet
    dfs_list.append(df)
jasmine = pd.concat(dfs_list)

#adding a column just to keep track of things
jasmine['directory'] = 'Jasmine'

#Lowercasing colnames just to keep life simple in final concat
jasmine.columns = [name.lower() for name in jasmine.columns]

Magazine :  41
Political News :  62
Sports :  47
Newspapers :  60
Television :  42


### 2.2 Tidying up W3 directory dataset for collation

In [3]:
filepath = "./data/allnewslisturl.xlsx"
w3 = pd.read_excel(filepath)
w3.head(3)

,states,url
0,alabama,https://www.alexcityoutlook.com/
1,alabama,https://www.annistonstar.com/
2,alabama,https://www.al.com/birmingham/


In [4]:
#Getting statecodes
state_codes_path = "./data/us_state_codes.html" #Got it from https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=53971
state_codes = pd.read_html("./data/us_state_codes.html")[0]
state_codes.State = state_codes.State.str.lower()
state_codes.drop(['Code', 'Abbreviation'], inplace=True, axis = 1)

In [5]:
#Cleaning up state names for better merge
w3.states = w3.states.str.replace('-', ' ')

In [6]:
#Merging
w3 = w3.merge(state_codes, left_on = 'states', right_on = 'State', how='left')
w3.drop(['states'], axis = 1, inplace = True)
w3.loc[w3['Alpha code'].isna(), :].State.unique()

array([], dtype=object)

In [7]:
#adding a column just to keep track of things
w3['directory'] = 'w3'

#Lowercasing colnames just to keep life simple in final concat
w3.columns = [name.lower() for name in w3.columns]

### 2.3 Tidying up USNPL directory dataset for collation

In [8]:
filepath = "./data/USNPL_newswebsite_urls.txt"
usnpl = pd.read_csv(filepath, sep='|')
usnpl.head(3)

,State,Newspaper URL
0,AL,http://www.sandmountainreporter.com/
1,AL,http://www.alexcityoutlook.com/
2,AL,http://www.andalusiastarnews.com/


In [9]:
#Getting statenames
usnpl = usnpl.merge(state_codes, left_on = 'State', right_on = 'Alpha code', how='left')
usnpl.drop(['State_x'], inplace=True, axis = 1)

In [10]:
#adding a column just to keep track of things
usnpl['directory'] = 'usnpl'

#Lowercasing colnames just to keep life simple in final concat
usnpl.columns = [name.lower() for name in usnpl.columns]

#Name cleanup for final concat to work
usnpl.columns = w3.columns

usnpl.head(3)

,url,state,alpha code,directory
0,http://www.sandmountainreporter.com/,alabama,AL,usnpl
1,http://www.alexcityoutlook.com/,alabama,AL,usnpl
2,http://www.andalusiastarnews.com/,alabama,AL,usnpl


### 2.4 Collating all 3

In [11]:
collated_df = pd.concat([jasmine, w3, usnpl])

## 3. Extract base urls of all these urls.

In [12]:
#Basic cleanup of url column
collated_df.url = collated_df.url.str.strip()
collated_df.tail(3)

,url,topic,directory,state,alpha code
5436,http://www.torringtontelegram.com/,NaN,usnpl,wyoming,WY
5437,http://www.pcrecordtimes.com/,NaN,usnpl,wyoming,WY
5438,http://www.wyodaily.com/,NaN,usnpl,wyoming,WY


In [13]:
def remove_www(x):
    if x.startswith('www') or x.startswith('ww1'):
        return x[4:]
    else:
        return x

In [14]:
collated_df['base'] = collated_df['url'].apply(lambda x: urlparse(x).netloc)
collated_df['base'] = collated_df['base'].apply(lambda x: remove_www(x))
collated_df.head(5)

,url,topic,directory,state,alpha code,base
0,https://beezzly.com/,Magazine,Jasmine,NaN,NaN,beezzly.com
1,https://www.elle.com/,Magazine,Jasmine,NaN,NaN,elle.com
2,https://www.gaming.net,Magazine,Jasmine,NaN,NaN,gaming.net
3,https://pingbacklinks.com,Magazine,Jasmine,NaN,NaN,pingbacklinks.com
4,https://www.inthesnow.com/,Magazine,Jasmine,NaN,NaN,inthesnow.com


In [15]:
collated_df.to_csv("./data/raw_data.txt", sep='|', index=False)

## 4. Enrich data of a base url if it is in multiple directories 
(i.e) if one directory has state info and another has topic info, then make sure the variables are populated in all rows against that url.

In the process of doing this I wanna make sure I don't generate additional duplicates. So I am going to declare a function to keep an eye on the dupes. This will also help me when I actively dedupe in the next step.

In [16]:
def var_dupe_counter(df, var, lvl_var = 'base'):
    counts = df.groupby(lvl_var).nunique()
    multi_var_urls = list(counts.loc[counts[var] > 1].index)
    multi_var_urls_df = df.loc[(df[lvl_var]).isin(multi_var_urls),:].sort_values(lvl_var)
    multi_var_urls_count_df = pd.DataFrame(multi_var_urls_df.groupby(lvl_var).count()[var]).reset_index()

    print(f"No of dupes across {var}:", len(multi_var_urls))
    print("No of rows:", len(multi_var_urls_df))
    print("Breakdown on no of rows: \n", pd.DataFrame(multi_var_urls_count_df.groupby(var).count()).reset_index())
    print("The df is returned if you want it...")
    return multi_var_urls_df

In [17]:
status_df = var_dupe_counter(collated_df, var = 'directory')

No of dupes across directory: 2184
No of rows: 5174
Breakdown on no of rows: 
     directory  base
0           2  1908
1           3   146
2           4    55
3           5    23
4           6    11
5           7     9
6           8     5
7           9     9
8          10     4
9          11     1
10         12     1
11         13     4
12         14     1
13         15     1
14         16     1
15         17     1
16         21     1
17         38     1
18         43     1
19         56     1
The df is returned if you want it...


In [18]:
status_df = var_dupe_counter(collated_df, var = 'state')

No of dupes across state: 55
No of rows: 313
Breakdown on no of rows: 
     state  base
0       2    12
1       3    13
2       4    14
3       5     3
4       6     5
5       7     1
6       9     2
7      10     1
8      11     1
9      21     1
10     36     1
11     43     1
The df is returned if you want it...


In [19]:
#Populating state everywhere I can
nonull_state = collated_df.loc[~collated_df['state'].isna(), ['base', 'state']]
nonull_state = nonull_state.drop_duplicates()
collated_df2 = collated_df.merge(nonull_state, how='left', left_on='base', right_on ='base')

#Removing additional column and renaming it
collated_df2 = collated_df2.drop(['state_x'], axis = 1) 
ind = list(collated_df2.columns).index('state_y')
colnames = list(collated_df2.columns)
colnames[ind] = 'state'
collated_df2.columns = colnames

#Removing obvious dupes
collated_df2 = collated_df2.drop_duplicates()

#Checking if nulls decreased
print("Current nulls: ", len(collated_df2.loc[collated_df2.state.isna(),:]), 
      "\nEarlier nulls: ", len(collated_df.loc[collated_df.state.isna(),:]))

Current nulls:  192 
Earlier nulls:  252


In [20]:
#Populating topic everywhere I can
nonull_topic = collated_df2.loc[~collated_df2['topic'].isna(), ['base', 'topic']]
nonull_topic = nonull_topic.drop_duplicates()
collated_df3 = collated_df2.merge(nonull_topic, how='left', left_on='base', right_on ='base')

#Removing additional column and renaming it
collated_df3 = collated_df3.drop('topic_x', axis = 1)
ind = list(collated_df3.columns).index('topic_y')
colnames = list(collated_df3.columns)
colnames[ind] = 'topic'
collated_df3.columns = colnames

#Removing obvious dupes
collated_df3 = collated_df3.drop_duplicates()

#Checking if nulls decreased
print("Current nulls: ", len(collated_df3.loc[collated_df3.topic.isna(),:]), 
      "\nEarlier nulls: ",len(collated_df2.loc[collated_df2.topic.isna(),:]))

Current nulls:  9464 
Earlier nulls:  9650


In [21]:
#State code issue
collated_df3[['state', 'alpha code']].drop_duplicates().dropna()

,state,alpha code
248,alabama,AL
256,florida,AL
263,georgia,AL
337,alaska,AK
369,arizona,AZ
...,...,...
9837,oregon,WI
9838,pennsylvania,WI
9839,tennessee,WI
9840,texas,WI


In [22]:
#Dealing with state code issue
collated_df3 = collated_df3.merge(state_codes, how = 'left', left_on='state', right_on = 'State')
collated_df3.drop(['alpha code', 'State'], axis=1, inplace=True)
collated_df3.columns = [col.lower() for col in collated_df3.columns]
collated_df3 = collated_df3.drop_duplicates()

In [23]:
#State code issue resolved
collated_df3[['state', 'alpha code']].drop_duplicates().dropna()

,state,alpha code
30,new york,NY
37,virginia,VA
43,district of columbia,DC
94,massachusetts,MA
121,california,CA
149,georgia,GA
150,maryland,MD
151,illinois,IL
152,indiana,IN
153,ohio,OH


In [24]:
status_df = var_dupe_counter(collated_df3, var = 'directory')
#Same no of dupes just more rows...

No of dupes across directory: 2184
No of rows: 6254
Breakdown on no of rows: 
     directory  base
0           2  1949
1           3   100
2           4    61
3           5    17
4           6     9
5           7     7
6           8    10
7           9     5
8          10     4
9          11     1
10         12     5
11         13     3
12         14     1
13         15     2
14         16     2
15         17     1
16         18     1
17         20     2
18         38     1
19         56     1
20         72     1
21       1032     1
The df is returned if you want it...


In [25]:
status_df = var_dupe_counter(collated_df, var = 'state')
#Same number of dupes but lesser number of rows

No of dupes across state: 55
No of rows: 313
Breakdown on no of rows: 
     state  base
0       2    12
1       3    13
2       4    14
3       5     3
4       6     5
5       7     1
6       9     2
7      10     1
8      11     1
9      21     1
10     36     1
11     43     1
The df is returned if you want it...


## 5. Dedupe the base urls across directories. This creates the "urls list".
    - To keep track if certain urls exist across states, a separate "collated dataset" is created which has states data and n repeats for n states.

In [26]:
#Dupes status across directories
status_df = var_dupe_counter(collated_df3, 'directory', lvl_var = 'base')

No of dupes across directory: 2184
No of rows: 6254
Breakdown on no of rows: 
     directory  base
0           2  1949
1           3   100
2           4    61
3           5    17
4           6     9
5           7     7
6           8    10
7           9     5
8          10     4
9          11     1
10         12     5
11         13     3
12         14     1
13         15     2
14         16     2
15         17     1
16         18     1
17         20     2
18         38     1
19         56     1
20         72     1
21       1032     1
The df is returned if you want it...


In [27]:
#Dupes status across states
status_df = var_dupe_counter(collated_df3, 'state', lvl_var = 'base')

No of dupes across state: 55
No of rows: 1493
Breakdown on no of rows: 
     state  base
0       2     4
1       4    26
2       6     4
3       8     6
4      10     2
5      12     4
6      14     1
7      15     1
8      16     2
9      18     1
10     20     1
11     38     1
12     72     1
13   1032     1
The df is returned if you want it...


In [28]:
#Checking in on the crazy 1032 dupes url...
print((collated_df3.groupby('base').count()['state'] == 1032).loc[lambda x: x == True])
#Writing it out to look at all 1032 rows
collated_df3.loc[collated_df3['base'] == 'bizjournals.com'].to_csv('temp.txt', sep = '|', index=False)

#I got what is happening here. It is legit.

base
bizjournals.com    True
Name: state, dtype: bool


In [29]:
#Deduping across directories. To ensure states data isn't lost, it is included in drop logic...
deduped_collated_df = collated_df3.drop_duplicates(['base', 'state'])

In [30]:
#Dupes status across directories
status_df = var_dupe_counter(deduped_collated_df, 'directory', lvl_var = 'base')
#Perfect

No of dupes across directory: 0
No of rows: 0
Breakdown on no of rows: 
 Empty DataFrame
Columns: [directory, base]
Index: []
The df is returned if you want it...


In [31]:
#Dupes status across states
status_df = var_dupe_counter(deduped_collated_df, 'state', lvl_var = 'base')
#So state repeat data isn't lost. Good.

No of dupes across state: 55
No of rows: 135
Breakdown on no of rows: 
    state  base
0      2    52
1      3     1
2      4     1
3     24     1
The df is returned if you want it...


In [32]:
# Writing it out
deduped_collated_df['base'].drop_duplicates().to_csv('./data/urls_list.txt', sep='|', index=False)
deduped_collated_df.to_csv('./data/collated_data.txt', sep='|', index=False)

## 6. Check how many requests of `https://[base url]/ads.txt` are successful. The successful urls form the "valid urls" list

Since this involves multithreading, it is done in a separate python script - See `checking_for_adstxt.py`. The response codes are outputted by that script.

In [33]:
response_codes = pd.read_csv('./data/responses.txt', sep='|', names = ['full_url', 'response'])
response_codes.head(5)

,full_url,response
0,https://okmagazine.com/ads.txt,200
1,https://failuremag.com/ads.txt,"HTTPSConnectionPool(host='failuremag.com', por..."
2,https://pingbacklinks.com/ads.txt,200
3,https://elle.com/ads.txt,200
4,https://menswearstyle.co.uk/ads.txt,403


In [34]:
valid_urls = response_codes.loc[(response_codes['response'] == '200')]['full_url'].apply(lambda x: urlparse(x).netloc)
valid_urls.name = 'base'
valid_urls.to_csv('./data/valid_urls.txt', sep='|', index=False)

## 7. Out of the "valid urls" account for redirects.
    - If the base url has a valid redirect to an actual ads.txt file, then that base url enters the "valid redirects" list .
    - If it doesn't lead to an ads.txt directly, but instead to a new base url, then check if that new base url leads to a valid ads.txt. If it does, then create a mapping file called `redirect mappings` from old to new base urls and this new base url should enter the "valid redirects list"

This needs parallelization. So it is done separately in a script - See `validating_redirects.py`

In [4]:
redirects_response = pd.read_csv('./data/redirects_response.txt', sep='|')
redirects_response.head(5)

,query_url,base_url,redirect,status
0,https://okmagazine.com/ads.txt,okmagazine.com,https://okmagazine.com/ads.txt,y
1,https://vanityfair.com/ads.txt,vanityfair.com,https://vanityfair.com/ads.txt,y
2,https://nytimes.com/ads.txt,nytimes.com,https://www.nytimes.com/ads.txt,y
3,https://pingbacklinks.com/ads.txt,pingbacklinks.com,https://pingbacklinks.com/ads.txt,y
4,https://thewalrus.ca/ads.txt,thewalrus.ca,https://thewalrus.ca/ads.txt,y


In [36]:
print(redirects_response.groupby('status').count()['base_url'])

#Ignoring the 2
redirects_response = redirects_response[redirects_response['status'] == 'y']

status
n       2
y    2894
Name: base_url, dtype: int64


In [37]:
#Does the base url have a valid redirect to an actual ads.txt?
redirects_response['valid_redirect'] = redirects_response['redirect'].apply(lambda x: x.__contains__('txt'))

In [38]:
redirects_response.groupby('valid_redirect').count()

,query_url,base_url,redirect,status
valid_redirect,,,,
False,378,378,378,378
True,2516,2516,2516,2516


In [39]:
def remove_ports(x):
    if x.__contains__(':'):
        x = x[:-4]
    return x

In [40]:
#Extracting base urls from the redirects
redirects_response['base_redirect'] = redirects_response['redirect'].apply(lambda x: urlparse(x).netloc)
redirects_response['base_redirect'] = redirects_response['base_redirect'].apply(lambda x: remove_www(x))
redirects_response['base_redirect'] = redirects_response['base_redirect'].apply(lambda x: remove_ports(x))
redirects_response.head(5)

,query_url,base_url,redirect,status,valid_redirect,base_redirect
0,https://okmagazine.com/ads.txt,okmagazine.com,https://okmagazine.com/ads.txt,y,True,okmagazine.com
1,https://vanityfair.com/ads.txt,vanityfair.com,https://vanityfair.com/ads.txt,y,True,vanityfair.com
2,https://nytimes.com/ads.txt,nytimes.com,https://www.nytimes.com/ads.txt,y,True,nytimes.com
3,https://pingbacklinks.com/ads.txt,pingbacklinks.com,https://pingbacklinks.com/ads.txt,y,True,pingbacklinks.com
4,https://thewalrus.ca/ads.txt,thewalrus.ca,https://thewalrus.ca/ads.txt,y,True,thewalrus.ca


In [41]:
#Getting invalid redirects wherein base url and base of redirect url are not the same
condition = ~(redirects_response['valid_redirect'] == True) & ~(redirects_response['base_redirect'] == 
                                                               redirects_response['base_url'])
redirects_response.loc[condition]

,query_url,base_url,redirect,status,valid_redirect,base_redirect
210,https://leanleft.com/ads.txt,leanleft.com,https://tortdeform.com/category/free-consultat...,y,False,tortdeform.com
228,https://thelafayettesun.com/ads.txt,thelafayettesun.com,https://wilcoxnewspapers.com/sun/,y,False,wilcoxnewspapers.com
327,https://gazettes.com/ads.txt,gazettes.com,https://www.presstelegram.com/the-grunion/,y,False,presstelegram.com
336,https://bhcourier.com/ads.txt,bhcourier.com,https://beverlyhillscourier.com/,y,False,beverlyhillscourier.com
393,https://gorizont.com/ads.txt,gorizont.com,https://view.publitas.com/hmg-llc/gorizont-44-...,y,False,view.publitas.com
...,...,...,...,...,...,...
2832,https://recordstar.com/ads.txt,recordstar.com,https://www.usatoday.com/,y,False,usatoday.com
2833,https://oregonherald.com/ads.txt,oregonherald.com,https://139.162.49.128/,y,False,139.162.49.128
2866,https://dptribune.com/ads.txt,dptribune.com,https://www.statesmanexaminer.com/dptribune/,y,False,statesmanexaminer.com
2892,https://planetjh.com/ads.txt,planetjh.com,https://buckrail.com,y,False,buckrail.com


In [42]:
#Writing it out for processing in script
redirects_response.loc[condition]['base_redirect'].drop_duplicates().to_csv('./data/redirect_urls.txt', sep='|', index = False)

For the `base_redirect`s in these records, I need to check if there is a valid `ads.txt`

Since this needs parallelization, it is in a separate script (similar to the one used earlier to check of `ads.txt`) - See `checking_for_adstxt2.py`

In [43]:
responses2 = pd.read_csv('./data/responses2.txt', sep='|')
responses2.head(5)

,query_url,base_redirect,response
0,https://presstelegram.com/ads.txt,presstelegram.com,200
1,https://4.qth.com/ads.txt,4.qth.com,"HTTPSConnectionPool(host='4.qth.com', port=443..."
2,https://chronicleonline.com/ads.txt,chronicleonline.com,200
3,https://usatoday.com/ads.txt,usatoday.com,200
4,https://inforum.com/ads.txt,inforum.com,200


In [44]:
responses2.groupby('response').count()
#So the 41 must enter the valid redirects list. I must also create the mapping file for them

,query_url,base_redirect
response,,
200,41,41
404,12,12
"HTTPSConnectionPool(host='4.qth.com', port=443): Max retries exceeded with url: /ads.txt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021443D52200>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))",1,1


In [45]:
#Creating mapping file
mappings = responses2.loc[responses2.response == '200'].merge(redirects_response.loc[condition], how = 'left', 
                            left_on = 'base_redirect', right_on = 'base_redirect')
mappings[['base_url', 'base_redirect']].drop_duplicates().to_csv('./data/redirect_mappings.txt', sep='|', index=False)

In [46]:
valid_redirects = redirects_response.loc[redirects_response['valid_redirect'] == True, 'base_url']
print(len(valid_redirects))
valid_redirects = valid_redirects.append(responses2.loc[responses2['response'] == '200', 'base_redirect'])
print(len(valid_redirects))

2516
2557


C:\Users\venki\AppData\Local\Temp\ipykernel_27184\2982554147.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  valid_redirects = valid_redirects.append(responses2.loc[responses2['response'] == '200', 'base_redirect'])


In [47]:
valid_redirects = pd.DataFrame(valid_redirects, columns = ['base_url'])
valid_redirects.to_csv('./data/valid_redirects.txt', sep='|', index=False)

In [48]:
len(valid_redirects)

2557

And with that the cleaning is all done.

I am going to do one final check if everything works fine in a script. See `checking_for_adstxt3.py`

In [49]:
final_check = pd.read_csv('./data/responses_final.txt', sep='|')
final_check

,query_url,base_redirect,response
0,https://okmagazine.com/ads.txt,okmagazine.com,200.0
1,https://vanityfair.com/ads.txt,vanityfair.com,200.0
2,https://drudgereport.com/ads.txt,drudgereport.com,200.0
3,https://pingbacklinks.com/ads.txt,pingbacklinks.com,200.0
4,https://thewalrus.ca/ads.txt,thewalrus.ca,200.0
...,...,...,...
2549,https://starrgennett.org/ads.txt,starrgennett.org,200.0
2550,https://gevelreinigingen.be/ads.txt,gevelreinigingen.be,200.0
2551,https://leaderunion.com/ads.txt,leaderunion.com,200.0
2552,https://waynecojournalbanner.com/ads.txt,waynecojournalbanner.com,200.0


In [50]:
final_check.groupby('response').count()

,query_url,base_redirect
response,,
200.0,2552,2552
502.0,1,1


Good enough. Moving on...